In [ ]:
Readable PDF: (pdf to text)
    1. pdftotext
    2. pypdf2
    3. camelot(tabular data)
    4. tabula-py(tabular data)
    
    
Non-readable: Scanned pdf
    pdf2image
    
image to text:
    OCR:
        1. Pytesseract (location)
        2. Amazon extract
        3. Google Vision
    

# Import Libraries

In [35]:
import pandas as pd
import numpy as np
import os
import glob
import re

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
Unstructured Data: pdf, Videos, Images, text

In [3]:
os.getcwd()

'G:\\Velocity 20 NOV\\04_06_Naive_Bayes'

In [4]:
folder_path = 'movie_reviews\\'
os.listdir(folder_path)

['negative', 'positive']

In [10]:
negative_folder_path = folder_path + 'negative'
positive_folder_path = folder_path + 'positive'
negative_text_files = glob.glob(f'{negative_folder_path}\\*.txt')
positive_text_files = glob.glob(f'{positive_folder_path}\\*.txt')

In [9]:
len(negative_text_files)

1000

In [11]:
len(positive_text_files)

1000

In [19]:
review_list = []
for file_name in negative_text_files:
    f = open(file_name)
    text = f.read()
    text = text.lower()
    text = re.sub('[^a-z]+', ' ', text)
    f.close()
    review_list.append(text)


for file_name in positive_text_files:
    f = open(file_name)
    text = f.read()
    text = text.lower()
    text = re.sub('[^a-z]+', ' ', text)
    f.close()
    review_list.append(text)

In [20]:
len(review_list)

2000

In [22]:
neg_target = np.zeros(len(negative_text_files), dtype = int)
pos_target = np.ones(len(positive_text_files),dtype = int)

In [25]:
y = np.append(neg_target,pos_target)
y = pd.Series(y)
y

0       0
1       0
2       0
3       0
4       0
       ..
1995    1
1996    1
1997    1
1998    1
1999    1
Length: 2000, dtype: int32

# TF-IDF Vectorizer

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
tfidf_vector = TfidfVectorizer(stop_words='english',max_df=0.95,min_df=0.05)
x_tfidf = tfidf_vector.fit_transform(review_list)
# x_tfidf
x = pd.DataFrame(x_tfidf.toarray(), columns= tfidf_vector.get_feature_names())
x

,ability,able,absolutely,act,acting,action,actor,actors,actress,actual,...,writers,writing,written,wrong,wrote,year,years,yes,york,young
0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.049035,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.045701,0.0,0.0,0.000000
1,0.00000,0.000000,0.0,0.0,0.164134,0.079639,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,0.00000,0.000000,0.0,0.0,0.056017,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.066405,0.068888,0.000000,0.000000,0.000000,0.0,0.0,0.057220
3,0.00000,0.099515,0.0,0.0,0.000000,0.000000,0.000000,0.079786,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.074194,0.000000,0.0,0.0,0.000000
4,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.116082,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.00000,0.000000,0.0,0.0,0.040102,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.059425,0.047539,0.000000,0.000000,0.113246,0.000000,0.0,0.0,0.000000
1996,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.103339,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1997,0.05055,0.000000,0.0,0.0,0.000000,0.090837,0.000000,0.000000,0.0,0.050657,...,0.0,0.000000,0.000000,0.000000,0.051091,0.000000,0.000000,0.0,0.0,0.063745
1998,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.049242,0.090272,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.041972,0.000000,0.0,0.0,0.000000


# TRain test Split

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,y , test_size=0.2, random_state=1, stratify=y)

# Model Training

## 1. GaussianNB()

In [34]:
gnb_model = GaussianNB()
gnb_model.fit(x_train,y_train)

GaussianNB()

In [37]:
## Testing Data Evaluation

y_pred = gnb_model.predict(x_test)

cnf_mat = confusion_matrix(y_test, y_pred)
print('Confusion Matrix :\n', cnf_mat)

acc_score = accuracy_score(y_test, y_pred)
print('Accuracy Score:', acc_score)

clf_report = classification_report(y_test,y_pred)
print('Classification Report:',clf_report)

Confusion Matrix :
 [[161  39]
 [ 58 142]]
Accuracy Score: 0.7575
Classification Report:               precision    recall  f1-score   support

           0       0.74      0.81      0.77       200
           1       0.78      0.71      0.75       200

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.76       400
weighted avg       0.76      0.76      0.76       400



In [38]:
## Training Data Evaluation

y_pred_train = gnb_model.predict(x_train)

cnf_mat = confusion_matrix(y_train, y_pred_train)
print('Confusion Matrix :\n', cnf_mat)

acc_score = accuracy_score(y_train, y_pred_train)
print('Accuracy Score:', acc_score)

clf_report = classification_report(y_train, y_pred_train)
print('Classification Report:',clf_report)

Confusion Matrix :
 [[707  93]
 [123 677]]
Accuracy Score: 0.865
Classification Report:               precision    recall  f1-score   support

           0       0.85      0.88      0.87       800
           1       0.88      0.85      0.86       800

    accuracy                           0.86      1600
   macro avg       0.87      0.86      0.86      1600
weighted avg       0.87      0.86      0.86      1600



## 2. BernoulliNB()

In [39]:
bnl_model = BernoulliNB()
bnl_model.fit(x_train,y_train)

BernoulliNB()

In [40]:
## Testing Data Evaluation

y_pred = bnl_model.predict(x_test)

cnf_mat = confusion_matrix(y_test, y_pred)
print('Confusion Matrix :\n', cnf_mat)

acc_score = accuracy_score(y_test, y_pred)
print('Accuracy Score:', acc_score)

clf_report = classification_report(y_test,y_pred)
print('Classification Report:',clf_report)

Confusion Matrix :
 [[162  38]
 [ 56 144]]
Accuracy Score: 0.765
Classification Report:               precision    recall  f1-score   support

           0       0.74      0.81      0.78       200
           1       0.79      0.72      0.75       200

    accuracy                           0.77       400
   macro avg       0.77      0.77      0.76       400
weighted avg       0.77      0.77      0.76       400



In [41]:
## Training Data Evaluation

y_pred_train = bnl_model.predict(x_train)

cnf_mat = confusion_matrix(y_train, y_pred_train)
print('Confusion Matrix :\n', cnf_mat)

acc_score = accuracy_score(y_train, y_pred_train)
print('Accuracy Score:', acc_score)

clf_report = classification_report(y_train, y_pred_train)
print('Classification Report:',clf_report)

Confusion Matrix :
 [[703  97]
 [174 626]]
Accuracy Score: 0.830625
Classification Report:               precision    recall  f1-score   support

           0       0.80      0.88      0.84       800
           1       0.87      0.78      0.82       800

    accuracy                           0.83      1600
   macro avg       0.83      0.83      0.83      1600
weighted avg       0.83      0.83      0.83      1600



## 3. MultinomialNB()

In [42]:
mlb_model = MultinomialNB()
mlb_model.fit(x_train,y_train)

MultinomialNB()

In [43]:
## Testing Data Evaluation

y_pred = mlb_model.predict(x_test)

cnf_mat = confusion_matrix(y_test, y_pred)
print('Confusion Matrix :\n', cnf_mat)

acc_score = accuracy_score(y_test, y_pred)
print('Accuracy Score:', acc_score)

clf_report = classification_report(y_test,y_pred)
print('Classification Report:',clf_report)

Confusion Matrix :
 [[163  37]
 [ 41 159]]
Accuracy Score: 0.805
Classification Report:               precision    recall  f1-score   support

           0       0.80      0.81      0.81       200
           1       0.81      0.80      0.80       200

    accuracy                           0.81       400
   macro avg       0.81      0.80      0.80       400
weighted avg       0.81      0.81      0.80       400



In [44]:
## Training Data Evaluation

y_pred_train = mlb_model.predict(x_train)

cnf_mat = confusion_matrix(y_train, y_pred_train)
print('Confusion Matrix :\n', cnf_mat)

acc_score = accuracy_score(y_train, y_pred_train)
print('Accuracy Score:', acc_score)

clf_report = classification_report(y_train, y_pred_train)
print('Classification Report:',clf_report)

Confusion Matrix :
 [[695 105]
 [126 674]]
Accuracy Score: 0.855625
Classification Report:               precision    recall  f1-score   support

           0       0.85      0.87      0.86       800
           1       0.87      0.84      0.85       800

    accuracy                           0.86      1600
   macro avg       0.86      0.86      0.86      1600
weighted avg       0.86      0.86      0.86      1600

